# Module 1, Section 2: Building Your First Agent

## Learning Goals

- Replace manual tool calling with the `create_agent` abstraction
- Add persistent memory to enable multi-turn conversations
- Understand threads for managing separate conversations
- See streaming responses in action

## Overview

In Section 1, we built the tool calling loop manually. While educational, that's a lot of tedious code!

In this section, we'll use `create_agent` - a powerful abstraction that:
- Handles the entire tool calling loop automatically
- Adds conversation memory out of the box
- Supports streaming for better UX
- Requires just a few lines of code

By the end, you'll see how `create_agent` replaces ~50 lines of manual code with ~5 lines!


## Setup

Load environment variables:

In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## 1. Import Tools

**Note on Refactoring:** In Section 1, we defined tools inline for learning purposes. Now that you understand how tools work, we've moved them to the shared `tools/` directory for reuse across multiple sections.

In [2]:
# Import our shared database tools
from tools import get_order_status, get_product_price

print("✓ Tools imported from tools/database.py!")
print(f"  - {get_order_status.name}")
print(f"  - {get_product_price.name}")

✓ Tools imported from tools/database.py!
  - get_order_status
  - get_product_price


## 2. Create Your First Agent

This is where the magic happens! Let's replace all that manual loop code with the `create_agent` abstraction which will run the loop for us:
1. Model decides which tool to call (if any)
2. Tool gets executed
3. Result goes back to model
4. Repeat until task is complete

The prebuilt agent handles running the loop described above - you just specify the system prompt and tools.


In [6]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

# Configure model
MODEL = "anthropic:claude-haiku-4-5"
llm = init_chat_model(MODEL)

# Create agent - THIS REPLACES ALL THE MANUAL LOOP CODE!
agent = create_agent(
    model=llm,
    tools=[get_order_status, get_product_price],
    system_prompt="You are a helpful customer support assistant for TechHub.",
)

Notice the benefits:
  - Automatic tool calling loop
  - No manual message management
  - Just ~5 lines of code!

Let's test it with the same query as Section 1:

In [11]:
result = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What's the status of order ORD-2024-0123?"}
        ]
    }
)

for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

What's the status of order ORD-2024-0123?
================================== Ai Message ==================================

[{'id': 'toolu_015SeiNz8w8WUR59qc3drmBL', 'input': {'order_id': 'ORD-2024-0123'}, 'name': 'get_order_status', 'type': 'tool_use'}]
Tool Calls:
  get_order_status (toolu_015SeiNz8w8WUR59qc3drmBL)
 Call ID: toolu_015SeiNz8w8WUR59qc3drmBL
  Args:
    order_id: ORD-2024-0123
================================= Tool Message =================================
Name: get_order_status

Order ORD-2024-0123: Status=Delivered, Shipped=2024-12-07, Tracking=1Z999AA113527782
================================== Ai Message ==================================

Great! Here's the status of your order:

**Order ID:** ORD-2024-0123
- **Status:** Delivered ✓
- **Shipped Date:** December 7, 2024
- **Tracking Number:** 1Z999AA113527782

Your order has been successfully delivered! If you have any other questions a

**Notice:** This works great for single queries! But try a follow-up question...

In [13]:
# Try a follow-up that references the previous query
result = agent.invoke(
    {"messages": [{"role": "user", "content": "When was it shipped?"}]}
)

result["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'd be happy to help you find out when your order was shipped! However, I need your order ID to look up that information.

Could you please provide your order ID? It typically looks something like "ORD-2024-0123".


⚠️  The agent doesn't remember the previous order! We need memory.

## 3. Add Short-term Memory with Checkpointer

Right now, each agent invocation is independent. Let's add **short-term memory** so the agent can maintain context across multi-turn conversations.

LangGraph uses **checkpointers** to save and restore state:

In [24]:
from langgraph.checkpoint.memory import MemorySaver

# Add checkpointer for memory
checkpointer = MemorySaver()

agent_with_memory = create_agent(
    model=llm,
    tools=[get_order_status, get_product_price],
    system_prompt="You are a helpful customer support assistant for TechHub.",
    checkpointer=checkpointer,  # This enables memory!
)

In addition, we also need to pass a `thread_id` when invoking the agent. The `thread_id` acts as a unique identifier for each conversation, allowing the agent to keep separate histories for different users or sessions.

In [25]:
import uuid

# Create a thread for this conversation - common practice to use a uuid
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

# Turn 1: Ask about an order
print("[Turn 1]")
result = agent_with_memory.invoke(
    {
        "messages": [
            {"role": "user", "content": "What's the status of order ORD-2024-0123?"}
        ]
    },
    config=config,
)
result["messages"][-1].pretty_print()

# Turn 2: Follow-up question (references "it" from previous turn)
print("\n[Turn 2]")
result = agent_with_memory.invoke(
    {"messages": [{"role": "user", "content": "When was it shipped?"}]},
    config=config,  # same thread_id
)
result["messages"][-1].pretty_print()

print("\n✓ The agent remembers we're talking about ORD-2024-0123!")

[Turn 1]
================================== Ai Message ==================================

Your order **ORD-2024-0123** has been **delivered**! 

Here are the details:
- **Status:** Delivered
- **Shipped Date:** December 7, 2024
- **Tracking Number:** 1Z999AA113527782

If you have any other questions about your order, feel free to ask!

[Turn 2]
================================== Ai Message ==================================

According to the order status, your order **ORD-2024-0123** was shipped on **December 7, 2024**.

✓ The agent remembers we're talking about ORD-2024-0123!


## 4. Thread Separation

Different `thread_id`s create separate conversations with isolated memory:

In [ ]:
# Thread 1: Customer A
config_user1 = {"configurable": {"thread_id": "customer-A"}}
result = agent_with_memory.invoke(
    {
        "messages": [
            {"role": "user", "content": "What's the status of order ORD-2024-0123?"}
        ]
    },
    config=config_user1,
)
print(f"[Customer A]: {result['messages'][-1].content[:80]}...\n")

# Thread 2: Customer B (different conversation)
config_user2 = {"configurable": {"thread_id": "customer-B"}}
result = agent_with_memory.invoke(
    {"messages": [{"role": "user", "content": "How much is the MacBook Air?"}]},
    config=config_user2,
)
print(f"[Customer B]: {result['messages'][-1].content[:80]}...\n")

# Back to Thread 1: Memory is preserved
result = agent_with_memory.invoke(
    {"messages": [{"role": "user", "content": "When was it shipped?"}]},
    config=config_user1,
)
print(f"[Customer A follow-up]: {result['messages'][-1].content[:80]}...")

print("\n✓ Thread 1 remembers order ORD-2024-0123, Thread 2 has no knowledge of it!")

[Customer A]: Great! Here's the status of your order:

**Order ID:** ORD-2024-0123
- **Status:...

[Customer B]: The **MacBook Air M2 (13-inch, 256GB)** is priced at **$1,199.00** and is curren...

[Customer A follow-up]: According to the order status I just looked up, your order **ORD-2024-0123 was s...

✓ Thread 1 remembers order ORD-2024-0123, Thread 2 has no knowledge of it!


### Key Takeaway:
- Checkpointers enable memory across interactions
- Thread IDs separate different conversations
- State persists automatically - no manual state management needed!

## 5. Streaming Responses for Better UX

LLMs can take a while to respond.

**Streaming** shows progress in real-time, dramatically improving user experience. To stream responses token-by-token - just change `.invoke()` to `.stream()`:

In [46]:
print("Streaming response:\n")

for message_chunk, metadata in agent.stream(
    {
        "messages": [
            {"role": "user", "content": "What's the status of order ORD-2024-0125?"}
        ]
    },
    stream_mode="messages",
):
    if metadata.get("langgraph_node") == "model":  # only print the model's output
        for block in message_chunk.content_blocks:
            if block.get("type") == "text" and block.get("text"):
                print(block.get("text"), end="", flush=True)

Streaming response:

Great! Here's the status of your order:

**Order ORD-2024-0125**
- **Status:** Delivered ✓
- **Shipped Date:** December 14, 2024
- **Tracking Number:** 1Z999AA127417599

Your order has been successfully delivered! If you have any questions about your delivery or need further assistance, feel free to ask.

> **Note:** You can control what types of information are streamed (e.g., messages, tool calls, etc.) by setting the `stream_mode` parameter. This allows fine-grained control over what you receive in real-time. See the [LangGraph streaming docs](https://docs.langchain.com/oss/python/langgraph/streaming) for details.


## Key Takeaways

### What We Learned

1. **`create_agent` is powerful** - Replaces ~50 lines of manual code with ~5 lines
2. **Checkpointers enable memory** - Add `checkpointer=MemorySaver()` for conversation memory
3. **Threads separate conversations** - Use unique `thread_id` for each user/conversation
4. **Streaming improves UX** - Just change `.invoke()` to `.stream()`

### What's Next

In **Section 3**, we'll build a **multi-agent system**:
- Specialized sub-agents (Database Agent, RAG Agent)
- Supervisor pattern for routing
- Coordinating complex workflows

The simple agent you built here becomes the building block for more sophisticated systems!